<a href="https://colab.research.google.com/github/JefNtungila/DS-Unit-2-Classification-2/blob/master/DS_Sprint_Challenge_9_Classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science, Unit 2_

# Classification 2 Sprint Challenge: Predict Chicago food inspections 🍔

For this Sprint Challenge, you'll use a dataset with information from inspections of restaurants and other food establishments in Chicago from January 2010 to March 2019. 

[See this PDF](https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF) for descriptions of the data elements included in this dataset.

According to [Chicago Department of Public Health — Food Protection Services](https://www.chicago.gov/city/en/depts/cdph/provdrs/healthy_restaurants/svcs/food-protection-services.html), "Chicago is home to 16,000 food establishments like restaurants, grocery stores, bakeries, wholesalers, lunchrooms, mobile food vendors and more. Our business is food safety and sanitation with one goal, to prevent the spread of food-borne disease. We do this by inspecting food businesses, responding to complaints and food recalls." 

#### Your challenge: Predict whether inspections failed

The target is the `Fail` column.

- When the food establishment failed the inspection, the target is `1`.
- When the establishment passed, the target is `0`.

#### Run this cell to load the data:

In [0]:
import pandas as pd

train_url = 'https://drive.google.com/uc?export=download&id=13_tP9JpLcZHSPVpWcua4t2rY44K_s4H5'
test_url  = 'https://drive.google.com/uc?export=download&id=1GkDHjsiGrzOXoF_xcYjdzBTSjOIi3g5a'

train = pd.read_csv(train_url)
test  = pd.read_csv(test_url)

assert train.shape == (51916, 17)
assert test.shape  == (17306, 17)

### Part 1: Preprocessing

You may choose which features you want to use, and whether/how you will preprocess them. If you use categorical features, you may use any tools and techniques for encoding. (Pandas, category_encoders, sklearn.preprocessing, or any other library.)

_To earn a score of 3 for this part, find and explain leakage. The dataset has a feature that will give you an ROC AUC score > 0.90 if you process and use the feature. Find the leakage and explain why the feature shouldn't be used in a real-world model to predict the results of future inspections._



In [2]:
train.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Violations,Latitude,Longitude,Location,Fail
0,2088270,"TOM YUM RICE & NOODLE, INC.",TOM YUM CAFE,2354911.0,Restaurant,Risk 1 (High),608 W BARRY,CHICAGO,IL,60657.0,2017-09-15T00:00:00,Canvass,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.938007,-87.644755,"{'longitude': '-87.6447545707008', 'latitude':...",1
1,555268,FILLING STATION & CONVENIENCE STORE,FILLING STATION & CONVENIENCE STORE,1044901.0,Grocery Store,Risk 3 (Low),6646-6658 S WESTERN AVE,CHICAGO,IL,60636.0,2011-10-20T00:00:00,Complaint Re-Inspection,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.772402,-87.683603,"{'longitude': '-87.68360273081268', 'latitude'...",0
2,1751394,A P DELI,A P DELI,47405.0,Restaurant,Risk 1 (High),2025 E 75TH ST,CHICAGO,IL,60649.0,2016-04-05T00:00:00,Canvass Re-Inspection,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.758779,-87.575054,"{'longitude': '-87.57505446746121', 'latitude'...",0
3,1763905,FRANK'S CHICAGO SHRIMP HOUSE,FRANK'S CHICAGO SHRIMP HOUSE,6414.0,Restaurant,Risk 2 (Medium),4459 S ARCHER AVE,CHICAGO,IL,60632.0,2016-04-29T00:00:00,Canvass,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.812181,-87.707125,"{'longitude': '-87.70712481334274', 'latitude'...",0
4,453326,MORRILL,MORRILL,24571.0,School,Risk 1 (High),6011 S Rockwell (2600W) AVE,CHICAGO,IL,60629.0,2011-01-10T00:00:00,Canvass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",NaN,NaN,NaN,0


In [3]:
train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Inspection ID,51916,NaN,NaN,NaN,1.33027e+06,620266,44250,679620,1.42423e+06,1.94695e+06,2.27971e+06
DBA Name,51916,17049,SUBWAY,654,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AKA Name,51293,16350,SUBWAY,802,NaN,NaN,NaN,NaN,NaN,NaN,NaN
License #,51911,NaN,NaN,NaN,1.54949e+06,901828,0,1.14074e+06,1.95862e+06,2.21551e+06,8.70061e+06
Facility Type,51692,329,Restaurant,34264,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Risk,51904,3,Risk 1 (High),37731,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Address,51916,13954,11601 W TOUHY AVE,741,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,51863,39,CHICAGO,51659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,51906,1,IL,51906,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zip,51890,NaN,NaN,NaN,60628.8,22.737,60007,60614,60625,60643,60827


In [4]:
train.dtypes

Inspection ID        int64
DBA Name            object
AKA Name            object
License #          float64
Facility Type       object
Risk                object
Address             object
City                object
State               object
Zip                float64
Inspection Date     object
Inspection Type     object
Violations          object
Latitude           float64
Longitude          float64
Location            object
Fail                 int64
dtype: object

In [0]:
# train['Inspection Date'] = pd.to_datetime(train['Inspection Date'])
# test['Inspection Date'] = pd.to_datetime(test['Inspection Date'])

In [0]:
# Train on the data.

target = 'Fail'
X = train.drop(columns=target)
y = train[target]

In [7]:
# Do train/validate/test 3-way split
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size = 0.25, stratify=y, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((38937, 16), (12979, 16), (38937,), (12979,))

In [0]:
!pip install category_encoders shap

### Part 2: Modeling

**Fit a model** with the train set. (You may use scikit-learn, xgboost, or any other library.) Use cross-validation or do a three-way split (train/validate/test) and **estimate your ROC AUC** validation score.

Use your model to **predict probabilities** for the test set. **Get an ROC AUC test score >= 0.60.**

_To earn a score of 3 for this part, get an ROC AUC test score >= 0.70 (without using the feature with leakage)._




In [9]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier

processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median')
)

X_train_processed = processor.fit_transform(X_train)
X_val_processed = processor.transform(X_val)

eval_set = [(X_train_processed, y_train), 
            (X_val_processed, y_val)]

model = XGBClassifier(n_estimators=1000, n_jobs=-1)
model.fit(X_train_processed, y_train, eval_set=eval_set, eval_metric='auc', 
          early_stopping_rounds=10)

[0]	validation_0-auc:0.624515	validation_1-auc:0.60395
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.624538	validation_1-auc:0.60395
[2]	validation_0-auc:0.63071	validation_1-auc:0.606923
[3]	validation_0-auc:0.639285	validation_1-auc:0.606678
[4]	validation_0-auc:0.655452	validation_1-auc:0.615132
[5]	validation_0-auc:0.655491	validation_1-auc:0.60708
[6]	validation_0-auc:0.659331	validation_1-auc:0.611533
[7]	validation_0-auc:0.660352	validation_1-auc:0.608807
[8]	validation_0-auc:0.676894	validation_1-auc:0.611457
[9]	validation_0-auc:0.673927	validation_1-auc:0.608732
[10]	validation_0-auc:0.673925	validation_1-auc:0.61148
[11]	validation_0-auc:0.678288	validation_1-auc:0.611608
[12]	validation_0-auc:0.677426	validation_1-auc:0.608857
[13]	validation_0-auc:0.677089	validation_1-auc:0.609486
[14]	validation_0-auc:0.680596	validation_1-auc:0.606757
Sto

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

### Part 3: Visualization

Make one visualization for model interpretation. (You may use any libraries.) Choose one of these types:

- Feature Importances
- Permutation Importances
- Partial Dependence Plot
- Shapley Values

_To earn a score of 3 for this part, make at least two of these visualization types._



### Part 4: Gradient Descent

Answer both of these two questions:

- What does Gradient Descent seek to minimize?
- What is the "Learning Rate" and what is its function?

One sentence is sufficient for each.

_To earn a score of 3 for this part, go above and beyond. Show depth of understanding and mastery of intuition in your answers._

In [0]:
# Gradient Descent seeks to minimize the figure goint to the limit of the minimum. It does so iteratively

# The iterations it takes is defined as the learning rate.
